<center>
<h1> INFB6073 - Machine Learning :: Actividad en clase 1 </h1>
<h2> Depto. de Informática y Computación || Primer semestre 2023 </h2>
</center>
<br>

# Problema

Utilice la base de datos weather.csv para generar un modelo de regresión lineal para los casos mencionados abajo. En cada caso debe realizar las siguientes etapas

>1. Utilice validación cruzada de 5 fold.
>1. Obtenga el modelo desde la metodología de regresión teórica.
>1. Obtenga el modelo desde la metodología de regresión desde sklearn.
 >1. Cree combinaciones de variables para mejorar su predicción.

Los casos a analizar son:


<center>
<table width="50%">
  <tr>
    <th scope="row"> </th>
    <th><font size="3">Inputs</th>
    <th><font size="3">Output</th>
  </tr>
  <tr>
    <td><strong> <font size="4">Caso 1 </strong>  </td>
    <td>MinTemp</td>
    <td>MaxTem</td>
  </tr>
  <tr>
    <td><strong> <font size="4">Caso 2 </strong> </td>
    <td>Rainfall<br> Evaporation<br>Pressure9am<br>Pressure3pm<br>Sunshine</td>
    <td>MaxTem</td>
  </tr>
</table>
</center>

---

# <center><font color='red'><strong>Solución y Explicación</strong></font></center>


## Cargar librerías

In [4]:
import pandas as pd
import io
import numpy as np

## Cargar base de datos
El archivo de datos se sube a colab con las librería ''files''. Luego se extraen los datos y se cargan en el entorno mediante la librería pandas.

In [6]:
# decargar datos desde https://utem.instructure.com/courses/8437/files/179741?wrap=1

from google.colab import files
uploaded = files.upload()

if list(uploaded.keys())[0].split('.')[-1] in ['csv','txt']: data = pd.read_csv(io.StringIO(uploaded['weather.csv'].decode("utf-8")))
else : print('data read problem')

data

Saving weather.csv to weather.csv


,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,8.0,24.3,0.0,3.4,6.3,NW,30.0,SW,NW,6.0,...,29,1019.7,1015.0,7,7,14.4,23.6,No,3.6,Yes
1,14.0,26.9,3.6,4.4,9.7,ENE,39.0,E,W,4.0,...,36,1012.4,1008.4,5,3,17.5,25.7,Yes,3.6,Yes
2,13.7,23.4,3.6,5.8,3.3,NW,85.0,N,NNE,6.0,...,69,1009.5,1007.2,8,7,15.4,20.2,Yes,39.8,Yes
3,13.3,15.5,39.8,7.2,9.1,NW,54.0,WNW,W,30.0,...,56,1005.5,1007.0,2,7,13.5,14.1,Yes,2.8,Yes
4,7.6,16.1,2.8,5.6,10.6,SSE,50.0,SSE,ESE,20.0,...,49,1018.3,1018.5,7,7,11.1,15.4,Yes,0.0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,9.0,30.7,0.0,7.6,12.1,NNW,76.0,SSE,NW,7.0,...,15,1016.1,1010.8,1,3,20.4,30.0,No,0.0,No
362,7.1,28.4,0.0,11.6,12.7,N,48.0,NNW,NNW,2.0,...,22,1020.0,1016.9,0,1,17.2,28.2,No,0.0,No
363,12.5,19.9,0.0,8.4,5.3,ESE,43.0,ENE,ENE,11.0,...,47,1024.0,1022.8,3,2,14.5,18.3,No,0.0,No
364,12.5,26.9,0.0,5.0,7.1,NW,46.0,SSW,WNW,6.0,...,39,1021.0,1016.2,6,7,15.8,25.9,No,0.0,No


# Repaso de contenidos

## Modelo de regresión teórico

Un modelo de regresión posee la siguiente forma:
$$ y = \theta_{0} + \theta_{1}x_{1} + \theta_{2}x_{2} + \cdots$$
donde $\theta_i,\ i=\{0,1,\cdots\}$ son los coeficientes que debemos buscar, y $x_{i},\ i=\{1,2,\cdots\}$ son las variables de nuestro problema. 

><mark><font color='green' face = "Lucida Console"><strong>NOTA</strong><br>Recuerde que $\theta_{0}$ esta multiplicada por la variable $x_0$ que es $1$</font></mark> 

Primero es necesario ordenar los datos en matrices de entrada ($X$) de **n** muestras/filas y **m** variables/columnas y salida ($Y$) de **n** muestras/filas y **1** variable/columna. En el caso de las entradas, debemos incluir un vector columnas de 1's para estimar el término de bias/intersección, es decir

`X = np.concatenate((np.ones(shape=X.shape[0],1), X), axis=1)`




In [7]:
n = data.shape[0] # Número de datos
input_variable_name = ['MinTemp']
output_variable_name = ['MaxTemp']
X = np.c_[np.ones(shape=(n,1)), data[input_variable_name].values]
Y = data[output_variable_name].values

Considerando que tenemos los datos en matrices, para obtener el modelo de regresión debemos ordenar matricialmente la ecuación anterior como:

\begin{align*}
y & =\theta_{0}+\theta_{1}x_{1}+\theta_{2}x_{2}+\cdots\\
y & =\left[1,x_{1},x_{2},\cdots\right]\left[\begin{array}{c}
\theta_{0}\\
\theta_{1}\\
\theta_{2}\\
\cdots
\end{array}\right]\\
Y & =X\Theta
\end{align*}

y para despejar $\Theta$ de la ecuación anterior debemos realizar los siguientes pasos

\begin{align*}
Y & =X\Theta & \longrightarrow & &\text{Multiplicar $X^{T}$ por la izquierda}\\
X^{T}Y & =X^{T}X\Theta & \longrightarrow & &\text{Multiplicar el inverso de $X^{T}X$ por la izquierda}\\
\left(X^{T}X\right)^{-1}\left(X^{T}Y\right) & =\left(X^{T}X\right)^{-1}\left(X^{T}X\right)\Theta & \longrightarrow & &\text{La multiplicación de $X^{T}X$ y su inverso genera la matriz identidad $I$}\\
\left(X^{T}X\right)^{-1}\left(X^{T}Y\right) & =I\Theta & \longrightarrow & &\text{Resultado Final. Despejamos $\Theta$}
\end{align*}

Corroborando las dimensiones de las matrices de cada uno de los pasos anteriores tenemos:

\begin{align*}
[n\times1] & =[n\times m][m\times1]\\{}
[m\times n][n\times1] & =[m\times n][n\times m][m\times1]\\
\left([m\times n][n\times m]\right)\left([m\times n][n\times1]\right) & =\left([m\times n][n\times m]\right)\left([m\times n][n\times m]\right)[m\times1]\\
\left([m\times m]\right)\left([m\times1]\right) & =[m\times m][m\times1]
\end{align*}


In [8]:
theta_theoretical = np.linalg.inv(X.T@X) @ (X.T@Y)
print(theta_theoretical)

[[14.48005288]
 [ 0.83547708]]


In [9]:
text1 = 'La ecuación final de regresión es:'
text2 = 'y_{%s} = %2.3f'%(output_variable_name[0], theta_theoretical[0])
for i in range(1,len(theta_theoretical)):
  text2 += ' + %2.3f X_{%s}'%(theta_theoretical[i],input_variable_name[i-1])
print(text1,'\n',text2)

La ecuación final de regresión es: 
 y_{MaxTemp} = 14.480 + 0.835 X_{MinTemp}


Ahora creamos un muestreo regular entre los valores mínimo y máximo de la variable MinTemp para graficar el modelo obtenido.
><mark><font color='green' face = "Lucida Console"><strong>NOTA</strong><br>Recuerde que una vez obtenido el muestreo regular debemos ordenar los datos como la matriz $X$, es decir [muestras $\times$ columna] y concatenar el vector de 1's</font></mark> 


In [10]:
Xtest = np.linspace(X[:,1].min(), X[:,1].max(), 1001).reshape(-1,1)
Xtest = np.concatenate((np.ones(shape=(Xtest.shape[0],1)), Xtest),axis=1)

Ahora evaluamos el modelo en el conjunto de test ($X_{test}$)
$$ Y_{test} = X_{test} \Theta$$

In [11]:
Ytest = Xtest @ theta_theoretical

Ahora graficamos los datos originales y los datos de test.

In [12]:
import plotly.graph_objects as go
text_title = '$ \text{' + text1 + ' } \\' + text2 + '$'
fig = go.Figure()
fig.add_trace(go.Scatter(y=Y[:,0], x=X[:,1],mode='markers', name='Datos originales'))
fig.add_trace(go.Scatter(y=Ytest[:,0], x=Xtest[:,1],mode='lines', name=r'Tendencia del modelo.<br>Datos test'))
fig.update_layout(title=f"{text_title!r}", xaxis_title=input_variable_name[0], yaxis_title=output_variable_name[0])

## Validación cruzada
Teniendo la metodología para obtener el modelo de regresión, ahora agreguemos la validación cruzada de 5 campos. 

><mark><font color='#00587C' face = "Lucida Console"><strong>NOTA</strong><br>Recuerde que la validación cruzada es utilizada para detectar el sobreajuste o la falta de generalización de un patrón. En general los datos de entrenamiento pueden estar sesgados antes nuevos datos, por lo que es necesario estimar el nivel de confianza de los modelos y sus predicciones.</font></mark> 

Una buena práctica en el análisis de datos es dividir el conjunto de datos en conjunto de entrenamiento (70%), validación (15%) y prueba (15%). El uso de cada conjunto de datos es el siguiente:
* **Conjunto de entrenamiento:** Se utiliza para entrenar el modelo.
* **Conjunto de validación:** Se utiliza para seleccionar hiper-parámetros del modelo.
* **Conjunto de prueba:** Se utiliza para evaluar el desempeño del modelo analizado. También este conjunto sirve para comparar diferentes modelos.

En el caso que los datos sean pocos, se utiliza la estrategia de validación cruzada.

La validación cruzada de _N-Folds_ genera N subconjuntos de la misma cantidad de datos del total de muestras de la base de datos. Los datos de cada subconjunto son seleccionados aleatoriamente. Una función que realiza esta selección aleatoria es `sklearn.model_selection.KFold`

><mark><font color='#00587C' face = "Lucida Console"><strong>NOTA</strong><br>`sklearn.model_selection.KFold` solo elige aleatoriamente las muestras si la opción `shuffle=True`, en caso contrario los _folds_ son elegidos en el orden de los datos originales. </font></mark>

En este ejercicio sólo se nos solicitó utilizar validación cruzada, por lo que podemos obtener los índices de los diferentes _folds_ o subgrupos como:

In [13]:
from sklearn.model_selection import KFold
import plotly.express as px
mcolors = px.colors.qualitative.Plotly

In [14]:
kf = KFold(n_splits=5, shuffle=False)
fig = go.Figure()
index = np.arange(X.shape[0]) + 1
for i, (train_index, test_index) in enumerate(kf.split(X)):
  fig.add_trace(go.Scatter(x=train_index, y=(i-0.1)*np.ones((len(train_index))), mode='markers', marker=dict(color=mcolors[i], symbol="circle"),            name='Fold %d train'%(i)))
  fig.add_trace(go.Scatter(x=test_index,  y=(i+0.1)*np.ones((len(test_index))),  mode='markers', marker=dict(color=mcolors[i], symbol="diamond-tall-open"), name='Fold %d test'%(i)))
fig.update_layout(title=r'Índice de muestras pertenecientes a cada <i>fold</i> <br><b>(Selección SIN aleatoriedad shuffle=False)</b>', 
                  xaxis_title='Índice de las muestras', yaxis_title='Número de Fold')
fig.update_yaxes(tickvals=np.arange(0,5))

In [15]:
kf = KFold(n_splits=5, shuffle=True)
fig = go.Figure()
index = np.arange(X.shape[0]) + 1
for i, (train_index, test_index) in enumerate(kf.split(X)):
  fig.add_trace(go.Scatter(x=train_index, y=(i-0.1)*np.ones((len(train_index))), mode='markers', marker=dict(color=mcolors[i], symbol="circle"),            name='Fold %d train'%(i)))
  fig.add_trace(go.Scatter(x=test_index,  y=(i+0.1)*np.ones((len(test_index))),  mode='markers', marker=dict(color=mcolors[i], symbol="diamond-tall-open"), name='Fold %d test'%(i)))
fig.update_layout(title=r'Índice de muestras pertenecientes a cada <i>fold</i> <br><b>(Selección CON aleatoriedad shuffle=True)</b>', 
                  xaxis_title='Índice de las muestras', yaxis_title='Número de Fold')
fig.update_yaxes(tickvals=np.arange(0,5))

## Regresión usando libreria sklearn

Para realizar el modelo de regresión usando esta librería sólo debemos ordenar los datos de entrada en una matriz de **n** muestras/filas y **m** columnas/variables. A diferencia del modelo de regresión teórico analizado arriba, acá no es necesario concatenar una columna de 1's para obtener el bias/offset/intersección, ya que sklearn lo incluye por defecto. La función que debemos utilizas es [`sklearn.linear_model.LinearRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html).

Nuevamente generamos nuestras matrices $X$ e $Y$, pero sin incluir la columna de 1's en $X$.

In [16]:
n = data.shape[0] # Número de datos
input_variable_name = ['MinTemp']
output_variable_name = ['MaxTemp']
X = data[input_variable_name].values
Y = data[output_variable_name].values

y ahora obtenemos nuestro modelo de regresión lineal usando `sklearn`

In [17]:
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(X, Y)

theta_sklearn  = np.array([model.intercept_] + [temp for temp in model.coef_]).reshape(-1,1)
theta_sklearn

array([[14.48005288],
       [ 0.83547708]])

In [18]:
text1 = 'La ecuación final de regresión usando sklearn es:'
text2 = 'y_{%s} = %2.3f'%(output_variable_name[0], theta_sklearn[0])
for i in range(1,len(theta_theoretical)):
  text2 += ' + %2.3f X_{%s}'%(theta_sklearn[i],input_variable_name[i-1])
print(text1,'\n',text2)

La ecuación final de regresión usando sklearn es: 
 y_{MaxTemp} = 14.480 + 0.835 X_{MinTemp}


Ahora evaluamos el modelo en el conjunto de test ($X_{test}$)

$$ Y_{test} = model.predict(X_{test})$$



In [19]:
Xtest = np.linspace(X[:,0].min(), X[:,0].max(), 1001).reshape(-1,1)
Ytest = model.predict(Xtest)

Ahora graficamos los datos originales y los datos de test.

In [20]:
text_title = '$ \text{' + text1 + ' } \\' + text2 + '$'
fig = go.Figure()
fig.add_trace(go.Scatter(y=Y[:,0], x=X[:,0],mode='markers', name='Datos originales'))
fig.add_trace(go.Scatter(y=Ytest[:,0], x=Xtest[:,0],mode='lines', name=r'Tendencia del modelo.<br>Datos test'))
fig.update_layout(title=f"{text_title!r}", xaxis_title=input_variable_name[0], yaxis_title=output_variable_name[0])

In [21]:
def funcionParaCrearStringLatexCombinacionesVariables(nvar=3, npol=2):
  M=np.mgrid[[slice(0,npol+1) for _ in range(nvar)]].reshape(nvar,-1).T[:,nvar::-1]
  dd='1'
  for i in range(1,npol+1):
    f=M[M.sum(1)==i,:]
    for j in f:
      ff=''
      for idx_var, k in enumerate(j):
        if k!=0:
          if k==1:ff += 'X_{%d}'%(idx_var+1)
          else:   ff += 'X_{%d}^{%d}'%(idx_var+1,k)
      dd += ' + ' + ff
  return dd

## Mejorar predicción usando combinación de variables

El objetivo de cualquier modelo de regresión (u otro modelo que veamos en el curso) es predecir correctamente los puntos de la variable dependiente  (eje y) desde los puntos de la variable independiente (eje x). Como se observa en las grafica anterior, los puntos azules están muy alejados de la línea roja (predicción de nuestro modelo), por lo que es necesario mejorar nuestra predicción.

Una estrategia utilizada es crear combinaciones de las variables existentes multiplicándolas entre ellas como la suma de polinomios (de las variables existentes) hasta orden $npol$. Si tenemos $m$ variables, matemáticamente podemos generar el conjunto de variables combinadas como:

\begin{align}
X_{ext}=\bigcup_{p=0}^{npol}\bigcup_{k_{1}+k_{2}+\cdots+k_{m}=p}\left\{ x_{1}^{k_{1}}x_{2}^{k_{2}}\cdots x_{m}^{k_{m}}\right\} 
\end{align}

donde $k_{i}, \forall i \in {1,\cdots, m}$ son los exponentes de cada variable $x_{i}$.

Si consideramos $m=2$ variables y generamos combinaciones de estas hasta un polinomio de orden $npol=3$ tenemos:

\begin{align}
X_{ext} & =\left\{ x_{1}^{0}x_{2}^{0},x_{1}^{1}x_{2}^{0},x_{1}^{0}x_{2}^{1},x_{1}^{2}x_{2}^{0},x_{1}^{1}x_{2}^{1},x_{1}^{0}x_{2}^{2},x_{1}^{3}x_{2}^{0},x_{1}^{2}x_{2}^{1},x_{1}^{1}x_{2}^{2},x_{1}^{0}x_{2}^{3}\right\} \\
 & =\left\{ 1,x_{1},x_{2},x_{1}^{2},x_{1}x_{2},x_{2}^{2},x_{1}^{3},x_{1}^{2}x_{2},x_{1}x_{2}^{2},x_{2}^{3}\right\} 
\end{align}

Es importante destacar que el polinomio $x_{1}^{0}x_{2}^{0}=1$, por lo que ya tenemos nuestro término de intersección de nuestra regresión lineal.

Para generar polinomios utilizaremos la librería [`sklearn.preprocessing.PolynomialFeatures`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html).

Nuevamente generamos nuestras matrices $X$ e $Y$, pero sin incluir la columna de 1's en $X$.


In [22]:
n = data.shape[0] # Número de datos
input_variable_name = ['MinTemp']
output_variable_name = ['MaxTemp']
X = data[input_variable_name].values
Y = data[output_variable_name].values

y generamos una combinación de la variable *MinTemp* con un polinomio de orden $2$

In [23]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)
X_ext = poly.fit_transform(X)

La matriz extendida es de 3 variables, i.e., $ \left\{ 1, X \left[ MinTemp \right], X \left[ MinTemp \right] ^{2}\right\} $.

Estimando nuevamente el modelo de regresión lineal con esta nueva base de datos tenemos:

><mark><font color='#00587C' face = "Lucida Console"><strong>NOTA</strong><br>Si utilizamos la base de datos extendida desde `sklearn.preprocessing.PolynomialFeatures` debemos quitar el cálculo de la intersección en `sklearn.preprocessing.LinearRegression`, es decir, ` LinearRegression(fit_intercept=False)` </font></mark>

In [36]:
model = LinearRegression(fit_intercept=False).fit(X_ext, Y)

theta_sklearn_comb  = np.array([temp for temp in model.coef_]).reshape(-1,1)
theta_sklearn_comb

array([[1.45730320e+01],
       [7.53280841e-01],
       [5.66545943e-03]])

In [43]:
text1 = 'La ecuación final de regresión usando sklearn con base extendida es:'
text2 = 'y_{%s} = %2.3f'%(output_variable_name[0], theta_sklearn_comb[0])
for i in range(1,len(theta_sklearn_comb)):
  text2 += ' + %2.3f X_{%s}^{%d}'%(theta_sklearn_comb[i],input_variable_name[0],i)
print(text1,'\n',text2)

La ecuación final de regresión usando sklearn con base extendida es: 
 y_{MaxTemp} = 14.573 + 0.753 X_{MinTemp}^{1} + 0.006 X_{MinTemp}^{2}


Ahora evaluamos el modelo en el conjunto de test ($X_{test}$)

$$ Y_{test} = model.predict(X_{test})$$


In [49]:
ns = 1001
Xtest_ext = np.c_[np.ones(shape=(ns,1)),
              np.linspace(X_ext[:,0].min(), X_ext[:,0].max(), ns).reshape(-1,1),
              np.linspace(X_ext[:,1].min(), X_ext[:,1].max(), ns).reshape(-1,1)]
Ytest_ext = model.predict(Xtest_ext)

Ahora graficamos los datos originales y los datos de test extendido.

In [65]:
text_title = '$ \text{' + text1 + ' } \\' + text2 + '$'
fig = go.Figure()
fig.add_trace(go.Scatter3d(z=Y[:,0], x=X_ext[:,1],y=X_ext[:,2],mode='markers', name='Datos originales'))
fig.add_trace(go.Scatter3d(z=Ytest_ext[:,0], x=Xtest_ext[:,1], y=Xtest_ext[:,2],line=dict(width=200),mode='lines', name=r'Tendencia del modelo.<br>Datos test'))
fig.update_layout(title=f"{text_title!r}", xaxis_title=input_variable_name[0], yaxis_title=output_variable_name[0])


Una vez realizado el resumen de contenidos, resolveremos la actividad en clase.

---
# <font color='blue'>Caso 1</font>

* Input: MinTemp
* Output: MaxTemp
---

In [66]:
n = data.shape[0] # Número de datos
input_variable_name = ['MinTemp']
output_variable_name = ['MaxTemp']

## Análisis con modelo teórico y 5CV

In [84]:
X = np.c_[np.ones(shape=(n,1)), data[input_variable_name].values]
Y = data[output_variable_name].values

error_theoretical = [] # Guarda los MSE de Ytest vs Ypred para cada fold
errstd_theoretical = [] # Guarda las desviaciones de MSE de Ytest vs Ypred para cada fold
theta_theoretical = [] # Guarda los thetas estimados en cada fold
ypred_theoretical = [] # Guarda las datos de prediccion de test en cada fold
yreal_theoretical = [] # Guarda las datos de test en cada fold
lsims_theoretical = [] # Guarda las datos de las rectas estimadas en cada fold 


kf = KFold(n_splits=5, shuffle=True)
xsims = np.c_[np.ones((100,1)), np.linspace(X[:,1].min(),X[:,1].max(),100)]
for i, (train_index, test_index) in enumerate(kf.split(X)):
  Xtrain, Ytrain = X[train_index], Y[train_index]
  Xtest,  Ytest  = X[test_index],  Y[test_index]
  theta_theoretical.append(np.linalg.inv(Xtrain.T@Xtrain) @ (Xtrain.T@Ytrain))
  yreal_theoretical.append(Ytest)
  ypred_theoretical.append(Xtest @ theta_theoretical[i])
  error_theoretical.append( np.mean( (yreal_theoretical[i] - ypred_theoretical[i])**2) )
  errstd_theoretical.append( np.std( (yreal_theoretical[i] - ypred_theoretical[i])**2) )
  lsims_theoretical.append( xsims@ theta_theoretical[i])


In [86]:
theta_theoretical

[array([[14.46193764],
        [ 0.83291235]]),
 array([[14.45804683],
        [ 0.84080502]]),
 array([[14.59022598],
        [ 0.81458395]]),
 array([[14.37305081],
        [ 0.83538167]]),
 array([[14.51535697],
        [ 0.85307097]])]

In [89]:
text_title = '$ \text{' + text1 + ' } \\' + text2 + '$'
fig = go.Figure()
fig.add_trace(go.Scatter(y=Y[:,0], x=X[:,1],mode='markers', name='Datos originales'))
for i in range(5):
  fig.add_trace(go.Scatter(x=xsims[:,1], 
                           y=lsims_theoretical[i].reshape(-1), 
                           mode='lines',
                           name='Modelo %d: MSE %2.3f +- %2.3f'%(i+1,error_theoretical[i],errstd_theoretical[i])))
  
fig.update_layout(title='Resultados de 5CV con modelo teórico', xaxis_title=input_variable_name[0], yaxis_title=output_variable_name[0])

## Análisis con modelo sklearn y 5CV

In [91]:
X = data[input_variable_name].values
Y = data[output_variable_name].values

error_sklearn = [] # Guarda los MSE de Ytest vs Ypred para cada fold
errstd_sklearn = [] # Guarda las desviaciones de MSE de Ytest vs Ypred para cada fold
theta_sklearn = [] # Guarda los thetas estimados en cada fold
ypred_sklearn = [] # Guarda las datos de prediccion de test en cada fold
yreal_sklearn = [] # Guarda las datos de test en cada fold
lsims_sklearn = [] # Guarda las datos de las rectas estimadas en cada fold 


kf = KFold(n_splits=5, shuffle=True)
xsims = np.linspace(X[:,0].min(),X[:,0].max(),100).reshape(-1,1)
for i, (train_index, test_index) in enumerate(kf.split(X)):
  Xtrain, Ytrain = X[train_index], Y[train_index]
  Xtest,  Ytest  = X[test_index],  Y[test_index]
  model = LinearRegression().fit(Xtrain, Ytrain)
  theta_sklearn.append(np.array([model.intercept_] + [temp for temp in model.coef_]).reshape(-1,1))
  yreal_sklearn.append(Ytest)
  ypred_sklearn.append(model.predict(Xtest))
  error_sklearn.append( np.mean( (yreal_sklearn[i] - ypred_sklearn[i])**2) )
  errstd_sklearn.append( np.std( (yreal_sklearn[i] - ypred_sklearn[i])**2) )
  lsims_sklearn.append( model.predict(xsims))

In [92]:
theta_sklearn

[array([[14.48672901],
        [ 0.82945733]]),
 array([[14.54450969],
        [ 0.82909854]]),
 array([[14.197246  ],
        [ 0.88458318]]),
 array([[14.50571023],
        [ 0.81959178]]),
 array([[14.66589525],
        [ 0.81362235]])]

In [94]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=Y[:,0], x=X[:,0],mode='markers', name='Datos originales'))
for i in range(5):
  fig.add_trace(go.Scatter(x=xsims[:,0], 
                           y=lsims_sklearn[i].reshape(-1), 
                           mode='lines',
                           name='Modelo %d: MSE %2.3f +- %2.3f'%(i+1,error_sklearn[i],errstd_sklearn[i])))
  
fig.update_layout(title='Resultados de 5CV con modelo sklearn', xaxis_title=input_variable_name[0], yaxis_title=output_variable_name[0])

## Análisis con modelo sklearn, combinación de variables (polinomio orden 3) y 5CV

In [110]:
X = data[input_variable_name].values
Y = data[output_variable_name].values

from sklearn.pipeline import make_pipeline
degree = 5

error_sklearn_plus_comb = [] # Guarda los MSE de Ytest vs Ypred para cada fold
errstd_sklearn_plus_comb = [] # Guarda las desviaciones de MSE de Ytest vs Ypred para cada fold
theta_sklearn_plus_comb = [] # Guarda los thetas estimados en cada fold
ypred_sklearn_plus_comb = [] # Guarda las datos de prediccion de test en cada fold
yreal_sklearn_plus_comb = [] # Guarda las datos de test en cada fold
lsims_sklearn_plus_comb = [] # Guarda las datos de las rectas estimadas en cada fold 


kf = KFold(n_splits=5, shuffle=True)
xsims = np.linspace(X[:,0].min(),X[:,0].max(),100).reshape(-1,1)
for i, (train_index, test_index) in enumerate(kf.split(X)):
  Xtrain, Ytrain = X[train_index], Y[train_index]
  Xtest,  Ytest  = X[test_index],  Y[test_index]
  model = make_pipeline(PolynomialFeatures(degree), LinearRegression(fit_intercept=False)).fit(Xtrain, Ytrain)
  theta_sklearn_plus_comb.append(np.array([temp for temp in model['linearregression'].coef_]).reshape(-1,1))
  yreal_sklearn_plus_comb.append(Ytest)
  ypred_sklearn_plus_comb.append(model.predict(Xtest))
  error_sklearn_plus_comb.append( np.mean( (yreal_sklearn_plus_comb[i] - ypred_sklearn_plus_comb[i])**2) )
  errstd_sklearn_plus_comb.append( np.std( (yreal_sklearn_plus_comb[i] - ypred_sklearn_plus_comb[i])**2) )
  lsims_sklearn_plus_comb.append( model.predict(xsims))
# model.get_feature_names_out

In [111]:
theta_sklearn_plus_comb

[array([[ 1.46461455e+01],
        [ 4.96405312e-01],
        [-2.83941145e-03],
        [ 1.24402604e-02],
        [-1.19736502e-03],
        [ 3.17180241e-05]]),
 array([[ 1.47609802e+01],
        [ 3.67470657e-01],
        [-3.35643069e-02],
        [ 2.43984007e-02],
        [-2.11310863e-03],
        [ 5.15942258e-05]]),
 array([[ 1.48234877e+01],
        [ 2.45641214e-01],
        [ 7.49891644e-03],
        [ 1.85923677e-02],
        [-1.78972643e-03],
        [ 4.53087977e-05]]),
 array([[ 1.47449210e+01],
        [ 3.34046713e-01],
        [ 6.67216544e-03],
        [ 1.71512236e-02],
        [-1.61922347e-03],
        [ 3.97174096e-05]]),
 array([[ 1.47093844e+01],
        [ 2.94599690e-01],
        [ 2.52968867e-02],
        [ 8.11915448e-03],
        [-5.96365960e-04],
        [ 7.58182788e-06]])]

In [112]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=Y[:,0], x=X[:,0],mode='markers', name='Datos originales'))
for i in range(5):
  fig.add_trace(go.Scatter(x=xsims[:,0], 
                           y=lsims_sklearn_plus_comb[i].reshape(-1), 
                           mode='lines',
                           name='Modelo %d: MSE %2.3f +- %2.3f'%(i+1,error_sklearn_plus_comb[i],errstd_sklearn_plus_comb[i])))
  
fig.update_layout(title='Resultados de 5CV con modelo sklearn + Combinacion de variables', xaxis_title=input_variable_name[0], yaxis_title=output_variable_name[0])

Para el caso 2 solo debe cambiar la cantidad de variables de entrada y usar los mismos códigos anteriores. 😄